In [ ]:
import os


def plot_image_ara(img_ara, xlabel, folder=None, title=None):
    rows=img_ara.shape[0]
    cols=img_ara.shape[1]

    print(rows,cols)

    f, axarr = plt.subplots(rows, cols, figsize=(cols, rows), squeeze=False)
    for c in range(cols):

        if c==0:
            # axarr[0, c].set_yticklabels( ['', '', 'Sex=0 \nEye=1'], rotation=0 ,fontsize=12 )
            # axarr[1, c].set_yticklabels( ['', '', 'Sex=1 \n Eye=1'], rotation=0 ,fontsize=12)
            for id, lb in enumerate(xlabel):
                axarr[id, c].set_yticklabels( ['', '', lb], rotation=0 ,fontsize=12 )



        for r in range(rows):
            if c>0:
                axarr[r, c].get_yaxis().set_ticks([])

            axarr[r, c].get_xaxis().set_ticks([])
            img= img_ara[r][c].cpu().detach().numpy()
            # img= np.transpose(img, (1,2,0))
            axarr[r, c].imshow(img)


        f.subplots_adjust(left=0, right=1, top=1, bottom=0, wspace=0, hspace=0)

    if folder==None:

        plt.show()
    else:
        os.makedirs(folder, exist_ok=True)
        plt.savefig(f'{folder}/{title}.png', bbox_inches='tight')

    plt.close()

In [ ]:
exp_name = "sex_eyeglass"
targetAtt = 'Eyeglasses'
plot_folder= '/local/scratch/a/rahman89/PycharmProjects/modularCelebA/modularTraining/PLOTS'
folder= f'/local/scratch/a/rahman89/PycharmProjects/modularCelebA/{exp_name}'

In [ ]:
import copy

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import copy
from torch.utils.data import Dataset, DataLoader
import torch
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
from modularTraining.constantFunctions import get_train_test_loaders, get_idata, denorm
from modularTraining.constantFunctions import get_sexWithAge

from Classifiers.mobileNet.model import MobileNet
from torch import nn
from Classifiers.mobileNet.train_valid_test import test

from modularTraining.constantFunctions import get_dataloaders
from Classifiers.mobileNet.train_valid_test import train, validation



intv_labels={}
male_list=[]
young_list=[]
img_list=[]

plt_img=[]


st=160
en = st+16
for m,y in zip([0,0,1,1], [0,1,0,1]):
    # ll, ii = get_idata(f'fakeIdom{m}y{y}/images_Idom{m}y{y}.pkl', f'fakeIdom{m}y{y}/labels_Idom{m}y{y}.pkl')  #getting  m0y0 data samples
    ll, ii = get_idata(folder, f'fakeIdom{m}e{y}/images_Idom{m}e{y}.pkl', f'fakeIdom{m}e{y}/labels_Idom{m}e{y}.pkl')  #getting  m0e0 data samples
    plt_img.append(denorm(ii['I'][st:en]).permute(0,2,3,1).unsqueeze(0))


    male_list.append(ll['Male'])
    young_list.append(ll['Young'])
    img_list.append(ii['I'])

#

xlabel=['Sex=0\nEye=0', 'Sex=0\nEye=1', 'Sex=1\nEye=0', 'Sex=1\nEye=1']
img_ara= torch.cat(plt_img)
# plot_image_ara(img_ara, '/local/scratch/a/rahman89/PycharmProjects/modularCelebA/modularTraining/PLOTS', f"eye1_sample{st}_{en}")
plot_image_ara(img_ara,xlabel, plot_folder , f"joint_sample{st}_{en}")

# P(Image) from train domain

In [ ]:
from modularTraining.constantFunctions import get_one_attribute_loader

exp_name = "sex_eyeglass"
dom_name = "domain1"
targetAtt = 'Eyeglasses'
lb_file = f'/local/scratch/a/rahman89/PycharmProjects/modularCelebA/{exp_name}/8_attribute_celeba_{dom_name}.pkl'
img_file = f'/local/scratch/a/rahman89/PycharmProjects/modularCelebA/{exp_name}/images_celeba_{dom_name}.pkl'
domain_dataset1, trainloader1, validloader1, testloader1 = get_train_test_loaders(targetAtt, lb_file, img_file)


eye_loader= get_one_attribute_loader(domain_dataset1, att1='Eyeglasses', val1=1, att2='Male', doNormalize=False)


# def get_one_attribute_loader(domain_dataset, att1, val1, att2, doNormalize=False):
#     retfo = (domain_dataset[att1] == val1)
#     att1_lb = domain_dataset[att2][retfo].reshape(-1, 1).type(torch.FloatTensor)
#     att1_image = domain_dataset['I'][retfo]
#     oldfemale_trainloader, oldfemale_validloader, rare_testloader = get_dataloaders(att1_lb, att1_image,
#                                                                                     split_t=0., split_v=0.,
#                                                                                     doNormalize=doNormalize)
#     return rare_testloader

In [ ]:
plt_img=[]

for img, lb in eye_loader:
    plt_img.append(denorm(img).permute(0,2,3,1))

rows= 8
plt_img = torch.cat(plt_img[0:80])
plt_img= plt_img.view(rows,-1, 224,224,3)
plot_image_ara(plt_img, ['' for r in range(rows)], plot_folder, "P(I|eye)")

In [ ]:
young=0
for img, lb in eye_loader:
    if lb==1:
        young+=1

print(young/ len(eye_loader))

In [ ]:
domain_dataset1.keys()